# KubeCon NA 2025 Demo - Cleanup

This notebook tears down the demo environment and cleans up all resources.

## What will be cleaned up:
- k3d cluster (including all deployments and data)
- Local cluster configuration
- Any running containers from the cluster

**Warning:** This operation is destructive and cannot be undone. Make sure you have saved any important data before proceeding.

In [1]:
import yaml
import os

# Load configuration
config_file = 'config.yaml'

if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    cluster_name = config['cluster']['name']
    print(f"Configuration loaded: Will delete cluster '{cluster_name}'")
else:
    cluster_name = 'kubecon-NA25'
    print(f"Config file not found, using default cluster name: '{cluster_name}'")

print("\n⚠️  WARNING: This will delete all cluster data!")
print(f"Cluster to be deleted: {cluster_name}")

Configuration loaded: Will delete cluster 'kubecon-NA25'

⚠️  WARNING: This will delete all cluster data!
Cluster to be deleted: kubecon-NA25


### Step 1: Check Current Cluster Status

Let's verify what clusters currently exist before deletion.

In [2]:
%%bash
echo "=== Current k3d Clusters ==="
k3d cluster list

echo ""
echo "=== Current kubectl context ==="
kubectl config current-context 2>/dev/null || echo "No active context"

=== Current k3d Clusters ===
NAME           SERVERS   AGENTS   LOADBALANCER
kubecon-NA25   1/1       0/0      true

=== Current kubectl context ===
k3d-kubecon-NA25


### Step 2: Optional - Export Resources

If you want to save any configurations before deletion, run this cell to export key resources.

In [3]:
%%bash
# Uncomment the following lines if you want to export resources

# EXPORT_DIR="exports/$(date +%Y%m%d_%H%M%S)"
# mkdir -p "$EXPORT_DIR"
#
# echo "Exporting resources to $EXPORT_DIR..."
#
# # Export Crossplane resources
# kubectl get providers -o yaml > "$EXPORT_DIR/providers.yaml" 2>/dev/null || true
# kubectl get compositions -o yaml > "$EXPORT_DIR/compositions.yaml" 2>/dev/null || true
# kubectl get xrds -o yaml > "$EXPORT_DIR/xrds.yaml" 2>/dev/null || true
#
# echo "✓ Resources exported to $EXPORT_DIR"

echo "Resource export is disabled by default. Uncomment the code above to enable."

Resource export is disabled by default. Uncomment the code above to enable.


### Step 3: Delete the k3d Cluster

This will remove the entire cluster and all its resources.

In [4]:
%%bash
set -e

CLUSTER_NAME="kubecon-NA25"

echo "=== Deleting k3d Cluster and Registry: $CLUSTER_NAME ==="

# Check if cluster exists
if k3d cluster list | grep -q "$CLUSTER_NAME"; then
    echo "Found cluster '$CLUSTER_NAME', deleting..."
    
    if k3d cluster delete "$CLUSTER_NAME"; then
        echo "✓ Cluster '$CLUSTER_NAME' deleted successfully"
    else
        echo "✗ Failed to delete cluster"
        exit 1
    fi
else
    echo "⚠ Cluster '$CLUSTER_NAME' not found (may already be deleted)"
fi

# Delete registry if it exists
echo ""
echo "Deleting local registry..."
if k3d registry list | grep -q "registry.localhost"; then
    if k3d registry delete registry.localhost; then
        echo "✓ Registry deleted successfully"
    else
        echo "⚠ Failed to delete registry"
    fi
else
    echo "⚠ Registry not found (may already be deleted)"
fi

echo ""
echo "Remaining k3d clusters:"
k3d cluster list

echo ""
echo "Remaining k3d registries:"
k3d registry list

=== Deleting k3d Cluster and Registry: kubecon-NA25 ===
Found cluster 'kubecon-NA25', deleting...
INFO[0000] Deleting cluster 'kubecon-NA25'              
INFO[0002] Deleting cluster network 'k3d-kubecon-NA25'  
INFO[0003] Deleting 1 attached volumes...               
INFO[0003] Removing cluster details from default kubeconfig... 
INFO[0003] Removing standalone kubeconfig file (if there is one)... 
INFO[0003] Successfully deleted cluster kubecon-NA25!   
✓ Cluster 'kubecon-NA25' deleted successfully

Deleting local registry...
✓ Registry deleted successfully

Remaining k3d clusters:
NAME   SERVERS   AGENTS   LOADBALANCER

Remaining k3d registries:
NAME   ROLE   CLUSTER   STATUS


### Step 4: Clean Up kubectl Context

Remove the cluster context from kubectl configuration.

In [5]:
%%bash
CLUSTER_NAME="kubecon-NA25"
CONTEXT_NAME="k3d-$CLUSTER_NAME"

echo "=== Cleaning up kubectl context ==="

# Delete context if it exists
if kubectl config get-contexts "$CONTEXT_NAME" &>/dev/null; then
    kubectl config delete-context "$CONTEXT_NAME" 2>/dev/null || true
    echo "✓ Context '$CONTEXT_NAME' removed"
else
    echo "⚠ Context '$CONTEXT_NAME' not found"
fi

# Delete cluster entry if it exists
if kubectl config get-clusters | grep -q "$CONTEXT_NAME"; then
    kubectl config delete-cluster "$CONTEXT_NAME" 2>/dev/null || true
    echo "✓ Cluster entry '$CONTEXT_NAME' removed"
fi

echo ""
echo "Current kubectl contexts:"
kubectl config get-contexts || echo "No contexts available"

=== Cleaning up kubectl context ===
⚠ Context 'k3d-kubecon-NA25' not found

Current kubectl contexts:
CURRENT   NAME   CLUSTER   AUTHINFO   NAMESPACE


### Step 5: Verification

Verify that cleanup was successful.

In [6]:
%%bash
echo "=== Cleanup Verification ==="
echo ""

echo "k3d clusters:"
k3d cluster list
echo ""

echo "kubectl contexts:"
kubectl config get-contexts 2>/dev/null || echo "No contexts"
echo ""

echo "Docker containers (k3d related):"
docker ps -a --filter "name=k3d" --format "table {{.Names}}\t{{.Status}}" 2>/dev/null || echo "No k3d containers"
echo ""

echo "✓ Cleanup verification complete"

=== Cleanup Verification ===

k3d clusters:
NAME   SERVERS   AGENTS   LOADBALANCER

kubectl contexts:
CURRENT   NAME   CLUSTER   AUTHINFO   NAMESPACE

Docker containers (k3d related):
NAMES     STATUS

✓ Cleanup verification complete


## Cleanup Complete!

The demo environment has been successfully torn down.

### What was cleaned up:
- ✓ k3d cluster deleted
- ✓ kubectl context removed
- ✓ Docker containers stopped and removed

### To start fresh:
Run the `00_Env-setup.ipynb` notebook again to recreate the environment.

### Remaining artifacts:
- Configuration files (config.yaml) - kept for reuse
- Setup manifests (setup/ directory) - kept for reuse
- Python environment and packages - kept for reuse

If you want to remove these as well, you can manually delete them from the file system.